In [1]:
import ast
from os.path import commonprefix
from urllib.parse import urlparse, parse_qs

import pandas as pd

In [2]:
""" Read sentence-level library """
sentences = pd.read_excel("sentences.xlsx")
sentences.head()

C:\Users\TomKaminski\Documents\AIEDN\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Name,Content,Tags,Metadata,Context,Color,Comment
0,"Ableiten Übersicht, Ableitung Funktionen, Diff...","Eine komplette Übersicht zum Thema Ableiten, A...","Differenziation, 11, Ableiten, Ableitung, Diff...","{'start': 0, 'end': 40, 'id': 'https://www.you...",NaN,NaN,NaN
1,"Ableiten Übersicht, Ableitung Funktionen, Diff...",Ich habe unter der Rubrik Analysis erstmal ein...,"Ableiten, Differenzieren, Ableitung, Differenz...","{'start': 7, 'end': 48, 'id': 'https://www.you...",NaN,NaN,NaN
2,"Ableiten Übersicht, Ableitung Funktionen, Diff...",Da geht es unter anderem um Differenzenprozent...,"11, Ableitung, Differenzieren, Ableiten, Diffe...","{'start': 13, 'end': 56, 'id': 'https://www.yo...",NaN,NaN,NaN
3,"Ableiten Übersicht, Ableitung Funktionen, Diff...",Da wird so die Einstimmung gemacht zu dem ganz...,"Ableitung, Differenzieren, 11, Differenziation...","{'start': 19, 'end': 62, 'id': 'https://www.yo...",NaN,NaN,NaN
4,"Ableiten Übersicht, Ableitung Funktionen, Diff...","Und jetzt kommt es darauf an, ob ihr in der Un...","Differenzieren, Ableitung, Ableiten, Differenz...","{'start': 26, 'end': 67, 'id': 'https://www.yo...",NaN,NaN,NaN


In [3]:
""" Equip sentences with unique video id """
def parse_url(metadata: str):
    # Get url from metadata dict
    return ast.literal_eval(metadata)["id"]

def parse_id(url: str):
    # parse id from youtube url
    parse_result = urlparse(url)
    query_params = parse_qs(parse_result.query)
    return query_params["v"][0]

print(parse_id("https://www.youtube.com/watch?v=5HnoTT7so8w&t=0s"))

sentences["Id"] = sentences["Metadata"].apply(lambda metadata: parse_id(parse_url(metadata)))
sentences[["Id", "Name"]].head()

5HnoTT7so8w


,Id,Name
0,5HnoTT7so8w,"Ableiten Übersicht, Ableitung Funktionen, Diff..."
1,5HnoTT7so8w,"Ableiten Übersicht, Ableitung Funktionen, Diff..."
2,5HnoTT7so8w,"Ableiten Übersicht, Ableitung Funktionen, Diff..."
3,5HnoTT7so8w,"Ableiten Übersicht, Ableitung Funktionen, Diff..."
4,5HnoTT7so8w,"Ableiten Übersicht, Ableitung Funktionen, Diff..."


In [4]:
""" Join all sentences belonging to the same video on new lines """
transcripts = sentences.groupby("Id").agg({
    "Name": lambda names: commonprefix(list(names))[:-1],
    "Content": "\n".join,
    "Tags": "first",
    "Metadata": "first"
})
transcripts.head()

,Name,Content,Tags,Metadata
Id,,,,
-1ecuhQJwNU,Graph zur Ableitung und Stammfunktion | Mathe ...,So der Graph der Ableitung und der Stammfunkti...,"Differenzieren, Differentiation, Differenziati...","{'start': 0, 'end': 46, 'id': 'https://www.you..."
-AyUqLeTtrE,"Tangente, Tangentengleichung aufstellen mittel...",Aufstellen von Tangentengleichungen.\nNicht ga...,"Bausteine allgemein+Sachzusammenhang, Kurvendi...","{'start': 0, 'end': 43, 'id': 'https://www.you..."
-PqSWya_6Vw,Ableitung Implizite Gleichung | Mathe by Danie...,Nehmen wir nochmal einen Ausdruck einer Gleich...,"Differenziation, Ableitung, Ableiten, Differen...","{'start': 0, 'end': 52, 'id': 'https://www.you..."
-TZPuJaJV1A,"Funktionen, E(x) Versionen, Wirtschaftsanwendu...",Da habe ich entsprechend jeweils eine Playlist...,"base, Polynomfunktionen, Analysis, Ganzrationa...","{'start': 11, 'end': 65, 'id': 'https://www.yo..."
-fk6qn0c_Jk,"Monotonie, Krümmung bei Funktionen, Übersicht ...",Hier ist wieder so ein faszinierendes Beispiel...,"base, Polynomfunktionen, Ganzrationale Funktio...","{'start': 1, 'end': 23, 'id': 'https://www.you..."


In [5]:
""" Clean the resulting data and tag the entries appropriately """
def delete_video_positions(metadata: str):
    dct = ast.literal_eval(metadata)
    del dct["start"]
    del dct["end"]
    return str(dct)

transcripts["Metadata"] = transcripts["Metadata"].apply(delete_video_positions)

transcripts["Tags"] = transcripts["Tags"].apply(lambda tags: "TRANSCRIPT_LEVEL, " + tags)
sentences["Tags"] = sentences["Tags"].apply(lambda tags: "SENTENCE_LEVEL, " + tags)

In [6]:
""" Merge the two libraries and dump the result to an xlsx file """
multi_level_library = pd.concat([transcripts, sentences]).reset_index(drop=True).drop("Id", axis=1)
display(multi_level_library)

multi_level_library.to_excel("transcripts_and_sentences.xlsx")

,Name,Content,Tags,Metadata,Context,Color,Comment
0,Graph zur Ableitung und Stammfunktion | Mathe ...,So der Graph der Ableitung und der Stammfunkti...,"TRANSCRIPT_LEVEL, Differenzieren, Differentiat...",{'id': 'https://www.youtube.com/watch?v=-1ecuh...,NaN,NaN,NaN
1,"Tangente, Tangentengleichung aufstellen mittel...",Aufstellen von Tangentengleichungen.\nNicht ga...,"TRANSCRIPT_LEVEL, Bausteine allgemein+Sachzusa...",{'id': 'https://www.youtube.com/watch?v=-AyUqL...,NaN,NaN,NaN
2,Ableitung Implizite Gleichung | Mathe by Danie...,Nehmen wir nochmal einen Ausdruck einer Gleich...,"TRANSCRIPT_LEVEL, Differenziation, Ableitung, ...",{'id': 'https://www.youtube.com/watch?v=-PqSWy...,NaN,NaN,NaN
3,"Funktionen, E(x) Versionen, Wirtschaftsanwendu...",Da habe ich entsprechend jeweils eine Playlist...,"TRANSCRIPT_LEVEL, base, Polynomfunktionen, Ana...",{'id': 'https://www.youtube.com/watch?v=-TZPuJ...,NaN,NaN,NaN
4,"Monotonie, Krümmung bei Funktionen, Übersicht ...",Hier ist wieder so ein faszinierendes Beispiel...,"TRANSCRIPT_LEVEL, base, Polynomfunktionen, Gan...",{'id': 'https://www.youtube.com/watch?v=-fk6qn...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
17392,Punkt und Koordinaten | Mathe by Daniel Jung_42,"Deshalb gut zu wissen, was ist überhaupt ein P...","SENTENCE_LEVEL, Funktionsarten, Funktionstypen...","{'start': 216, 'end': 246, 'id': 'https://www....",NaN,NaN,NaN
17393,Punkt und Koordinaten | Mathe by Daniel Jung_43,"Und wenn man noch weiter in die Tiefe will, vo...","SENTENCE_LEVEL, Funktionen Übersicht, base, Fu...","{'start': 220, 'end': 246, 'id': 'https://www....",NaN,NaN,NaN
17394,Punkt und Koordinaten | Mathe by Daniel Jung_44,Und ansonsten müsstet ihr hiermit gut durchkom...,"SENTENCE_LEVEL, Funktionen Übersicht, Funktion...","{'start': 230, 'end': 246, 'id': 'https://www....",NaN,NaN,NaN
17395,Punkt und Koordinaten | Mathe by Daniel Jung_45,"Ihr wisst, wenn doch noch irgendetwas ist, wen...","SENTENCE_LEVEL, Funktionen Übersicht, base, Fu...","{'start': 233, 'end': 246, 'id': 'https://www....",NaN,NaN,NaN
